In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calculate_angle(a,b,c):
  a = np.array(a)
  b = np.array(b)
  c = np.array(c)

  radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
  angle = np.abs(radians*180.0/np.pi)

  if angle>180.0:
    angle = 360 - angle

  return angle

In [43]:
cap = cv2.VideoCapture(1)

# Curl counter variable
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    ret, frame = cap.read()

    # Recolor image
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Make detection
    results = pose.process(image)
    # print(results)

    # Recolor image to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Extract landmarks
    try:
      landmarks = results.pose_landmarks.landmark
      # print(landmarks)

      # Get coordinates
      shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
      elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
      wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

      # Calculate angle
      angle = calculate_angle(shoulder, elbow, wrist)

      # Visualize angle
      cv2.putText(image, str(angle), 
                  tuple(np.multiply(elbow, [1920, 1080]).astype(int)), 
                  cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
      
      # Curl counter logic
      if angle>160:
        stage = 'down'
      if angle<30 and stage=='down':
        stage='up'
        counter+=1
        print(counter)

    except:
      pass

    # Render curl counter
    # setup status box
    cv2.rectangle(image, (0,0), (255,73), (245,117,16), -1)

    # Rep data
    cv2.putText(image, 'REPS', (15,12),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(counter), 
                  (10,60),
                  cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Stage data
    cv2.putText(image, 'STAGE', (65,12),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(stage), 
                  (60,60),
                  cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2),)
    
    cv2.imshow('Mediapipe feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  cap.release()
  cv2.destroyAllWindows()

1
2
3
4
5
6
7
8


In [30]:
np.multiply(elbow, [1920, 1080]).astype(int)

array([2174, 1048])

In [21]:
len(landmarks)

33

In [22]:
for lndmark in mp_pose.PoseLandmark:
  print(lndmark)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [23]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.92270595
y: 0.57175964
z: -0.12152221
visibility: 0.9879053

In [24]:
landmarks[0]

x: 0.6765014
y: 0.2449833
z: -0.9649048
visibility: 0.9999099

# Calculate Angles

In [25]:
def calculate_angle(a,b,c):
  a = np.array(a)
  b = np.array(b)
  c = np.array(c)

  radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
  angle = np.abs(radians*180.0/np.pi)

  if angle>180.0:
    angle = 360 - angle

  return angle

In [29]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [27]:
shoulder, elbow, wrist

([0.9227059483528137, 0.5717596411705017],
 [1.1327779293060303, 0.9712050557136536],
 [1.1423304080963135, 1.1282683610916138])

In [28]:
calculate_angle(shoulder, elbow, wrist)

155.74011075491225